In [1]:
from bs4 import BeautifulSoup
import requests
import re

class scrape():
scrape_genre: 地址(list of json file contains title and url, example: {"title":"玄幻","url":"https://my.qidian.com/bookcase?page={{page}}"}), 列表规则，书名规则，作者规则，分类规则，字数规则，简介规则，封面规则
scrape_book_detail: cover image, book_url, book name, author, 分类，字数，最新章节，简介
scrape_book_chapters: 章节规则, 章节名字规则，章节地址
scrape_content: 正文规则，标题规则


In [2]:
def return_result(result, content_type):
    if content_type == 'text':
        return result.text
    if content_type == 'href':
        return result.get('href')
    if content_type == 'src':
        return result.get('src')
    if content_type == 'textNodes':
        return result.textNodes
    if content_type == 'ownText':
        return result.ownText
    if content_type == '':
        return result


def convert_negative_index(n, length):
    if n < 0:
        return length + n
    return n


def parse_bracket_signs(location_str, arr_length):
    result = set()
    if '(' in location_str and ')' in location_str:
        locations = location_str.strip('()').split(',')
        for location in locations:
            if ':' in location:
                start, end = map(int, location.split(':'))
                start = convert_negative_index(start, arr_length)
                end = convert_negative_index(end, arr_length)
                result.update(range(start, end+1))
            else:
                location = convert_negative_index(int(location), arr_length)
                result.add(int(location))
    return result


def parse_location(location_str, arr):
    arr_length = len(arr)
    if '!' not in location_str:
        return list(parse_bracket_signs(location_str, arr_length)) if location_str else list(range(len(arr)))

    include_str, exclude_str = location_str.split('!', 1)
    include = parse_bracket_signs(
        include_str, arr_length) if include_str else set(range(len(arr)))
    exclude = parse_bracket_signs(
        exclude_str, arr_length) if exclude_str else set()

    return list(include - exclude)


""" def parse_location(location_str, arr):
    location = location_str.split('!')
    result = set(range(len(arr)))
    # there can only be up to 1 exclude. if there is more than 1, then it is invalid
    if len(location) <= 0 or len(location) > 2:
        return []
    
    # theres no exclude sign, means no !
    if len(location) == 1 and location[0] == '':
        return list(result)
    elif len(location) == 1 and location[0] != '':
        result = parse_bracket_signs(location[0])

    # there is exclude sign, means there is !
    if len(location) == 2:
        # if there is only exclude, means there is no include, '!(1-2)' or '!(1,2)'
        if location[0] == '' and location[1] != '':
            exclude = parse_bracket_signs(location[1])
            result = result - exclude
        
        # if there is both include and exclude, '(1-2)!(1-2)' or '(1,2)!(1,2)'
        elif location[0] != '' and location[1] != '':
            result = parse_bracket_signs(location[0])
            exclude = parse_bracket_signs(location[1])
            print('result: ', result)
            print('exclude: ', exclude)
            result = result - exclude
            print('result: ', result)
        
        # if there is only include, means there is no exclude, '(1-2)!' or '(1,2)!'
        elif location[0] != '' and location[1] == '':
            result = parse_bracket_signs(location[0])
        
        # if there no both
        else:
            return []
        
    return list(result) """


def get_content(content, rule):
    result = []
    for i in range(len(content)):
        res = content[i].find_all(rule['tag'], attrs=rule['attr'])
        location = parse_location(rule['location'], res)

        for j in location:

            result.append(return_result(res[j], rule['content']))
    return result


def scraper(url, rules):
    if len(rules) <= 0:
        return []
    r = requests.get(url)
    content = BeautifulSoup(r.text, 'html.parser')

    content = [content]
    for rule in rules:
        content = get_content(content, rule)

    return content

In [9]:
url = 'https://www.qidian.com/chapter/1038315291/771901670/'

rule = [{'tag': 'h1', 'attr': {}, 'location': '!', 'content': ''}]

scraper(url, rule)

[<h1 class="text-rh3 text-s-gray-900 font-medium mt-20px" data-v-b2769519="">我在超凡世界火炮洗地</h1>,
 <h1 class="title text-1.3em text-s-gray-900 leading-[1.35] font-medium break-all wrap-word inline-block r-font-black-medium" data-v-7415a5f2="">第1章：懦夫之举，绝不姑息！</h1>]

In [16]:
url2 = 'https://www.qidian.com/'

""" userinput = 'class.odd.0@tag.a.0@class.even.1'

section_rule = {'tag':'div', 'attr':{'class':'odd'}, 'location':'(0)', 'content':''}
children_rule = {'tag':'a', 'attr':{}, 'location':'(0)', 'content':'text'}
children2_rule = {'tag':'div', 'attr':{'class':'even'}, 'location':'(1)', 'content':'text'}

 """
section_rule = {'tag': 'dl', 'attr': {
    'class': 'cf'}, 'location': '(0)', 'content': ''}

children_rule = {'tag': 'a', 'attr': {'href': True, 'title': True,
                                      'target': True}, 'location': '', 'content': 'href'}

rule2 = [section_rule]
a = scraper(url2, rule2)

In [33]:
url = 'https://www.qidian.com/chapter/1035240682/730331779/'

section = {'tag': 'main', 'attr': {'id': 'c-730331779'}, 'location': '(0)', 'content': ''}
children = {'tag': 'p', 'attr': {}, 'location': '', 'content': 'text'}

rul3 = [section, children]

scraper(url, rul3)

['\u3000\u3000谷剑秋睁开了眼睛，刺眼的白灯一时模糊了他的视野。剧烈的耳鸣和脑中混乱的记忆碎片叫他忍不住干呕起来。',
 '\u3000\u3000“……”',
 '\u3000\u3000“哥？”',
 '\u3000\u3000正捂住胸口深呼吸的谷剑秋抬起头，弟弟谷天宇光脚站在门口，身上只穿着一条四角内裤，干瘪的身体露出肋骨，正不安地抿着嘴唇盯着自己。',
 '\u3000\u3000谷剑秋平顺呼吸，回答道：“我没事，把鞋穿上，给我倒杯热水。”',
 '\u3000\u3000“哦。”',
 '\u3000\u3000谷天宇快步跑开。',
 '\u3000\u3000“剑秋，剑秋，你又做噩梦啦？”',
 '\u3000\u3000母亲的声音隔着水泥墙传来。',
 '\u3000\u3000谷剑秋强打精神，提高了嗓门：“我没事，妈。睡觉翻身，没注意掉下去了。”',
 '\u3000\u3000他接过谷天宇手里滚烫的开水，揉了揉弟弟的满头碎发：“去睡吧。”',
 '\u3000\u3000目送弟弟离开，谷剑秋一口气把水喝干净。扯掉身上洗到掉色的蓝色毛毯，翻身下了床，趿着拖鞋走到阳台上，然后长长吁出一口气。',
 '\u3000\u3000纷冗的记忆碎片潮水般涌来。',
 '\u3000\u3000鬼子母，涌动的翼式神机，太空中漂浮的尸群，烧成残骸的姑射神人。',
 '\u3000\u3000那不是梦。',
 '\u3000\u3000他本来已经在冰冷的太空中昏死过去，结果苏醒过来就到了这儿。',
 '\u3000\u3000两百年前的旧海棠，两百年前的谷剑秋。',
 '\u3000\u3000今天是一八八四年五月二日。自己二十一岁，是江宁师范学堂博物科的两年生，家中有兄弟和姐姐，母亲瘫痪在床，一家五口人挤在江宁市天人坊一间六十平的旧洋楼里。',
 '\u3000\u3000上个月十号，哥哥谷西楼被人打死在鸡鸣寺街口，尸体倚在画满天女和佛陀的老旧手绘墙下面，成串的血珠从他湿漉漉的发丝末端滑落，和被开膛破肚流了满地的肝肠一起汇成大片的红泊。',
 '\u3000\u3000那一幕谷剑秋终身难忘。',
 '\u3000\u3000三个月后，天人坊突发火灾，火势绵延烧光了整栋楼，除了凑巧不在家的谷剑秋，母亲和姐弟三人都死在大火当中……',
 '\u3000\u3